In [1]:
import pandas as pd

In [2]:
from collections import defaultdict
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Autenticación
auth_manager = SpotifyClientCredentials(client_id="4fe8c3ad5e9c4851a77b6fbe2912fa5c", client_secret="62bbeef6c90d4de783fa926d0cd1b5a5")
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
#1.Definición y parámetros
#DE AQUÍ HASTA ABAJO SERÍA TODO FUNCIÓN DE EXTRACCIÓN DE DATOS.

def extraer_datos_por_genero_y_rango(genero, rango_anyo, resultados_por_anyo=200):
    albumes_por_anyo = defaultdict(list)
    canciones_por_anyo = defaultdict(list)

    # 2.Búsqueda de artistas, es el buccle principal
    for o in range(0, 200, 50):  # búsqueda de 200 artistas del género
    #este bucle va a iterar 4veces de 0 a50,50a100,100a150,150a200
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=o) #offset para paginar resultados
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']

            # 3.Extración álbumes por artista
            albumes_data = sp.artist_albums(artista_id, album_type=['album'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])
                    if album_lanzamiento in rango_anyo and len(albumes_por_anyo[album_lanzamiento]) < resultados_por_anyo:
                        albumes_por_anyo[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'id': album['id']
                        })
                except (KeyError, ValueError, TypeError):
                    continue
            
            # 4. Extracción de Canciones Top (+condición de parada para no"gastar"llamadas a la API)
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango_anyo and len(canciones_por_anyo[track_lanzamiento]) < resultados_por_anyo:
                            canciones_por_anyo[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id']
                            })
                    except (KeyError, ValueError, TypeError):
                        continue
            except 'SpotifyException':
                continue

    # 6. Devolver resultados en 2 listas separadas:
    return {
        'albumes': [{'tipo': 'album', **data} for album_lanzamiento in rango_anyo for data in albumes_por_anyo[album_lanzamiento][:resultados_por_anyo]],
        'canciones': [{'tipo': 'cancion', **data} for anyo in rango_anyo for data in canciones_por_anyo[anyo][:resultados_por_anyo]]
    }

In [4]:
#USO, llamadas:

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('country', range(2018, 2023))

In [5]:
albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # pedir lista de álbumes
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # pedir lista de canciones

In [6]:
type(tastyinfo_2018_2022)

dict

In [7]:
tastyinfo_2018_2022

{'albumes': [{'tipo': 'album',
   'artista': 'Eric Church',
   'genero': 'country',
   'nombre': 'Desperate Man',
   'año': 2018,
   'id': '5TjDN2hfsgNWVtP8Ew56Xx'},
  {'tipo': 'album',
   'artista': 'Jordan Davis',
   'genero': 'country',
   'nombre': 'Home State',
   'año': 2018,
   'id': '3CXa4AY5Rt4kymz1vW90WR'},
  {'tipo': 'album',
   'artista': 'Tyler Childers',
   'genero': 'country',
   'nombre': 'Live on Red Barn Radio I & II',
   'año': 2018,
   'id': '7lf0m3iNU59X4r8udCQcB6'},
  {'tipo': 'album',
   'artista': 'Jason Aldean',
   'genero': 'country',
   'nombre': 'Rearview Town',
   'año': 2018,
   'id': '0m3ZCK4dSvyq6KUzEWdMhE'},
  {'tipo': 'album',
   'artista': 'Thomas Rhett',
   'genero': 'country',
   'nombre': 'Life Changes (Deluxe Version)',
   'año': 2018,
   'id': '4aHakdIA1e8HoRRUsYszDY'},
  {'tipo': 'album',
   'artista': 'Luke Combs',
   'genero': 'country',
   'nombre': "This One's for You Too (Deluxe Edition)",
   'año': 2018,
   'id': '53Oa5Bu0UTU8o8qCTaHKoz'},

In [8]:
len(tastyinfo_2018_2022)

2

In [10]:
# Conversión a DataFrame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(df_2018_2022_album)

      tipo              artista   genero                              nombre  \
0    album          Eric Church  country                       Desperate Man   
1    album         Jordan Davis  country                          Home State   
2    album       Tyler Childers  country       Live on Red Barn Radio I & II   
3    album         Jason Aldean  country                       Rearview Town   
4    album         Thomas Rhett  country       Life Changes (Deluxe Version)   
..     ...                  ...      ...                                 ...   
489  album     Charley Crockett  country                   The Man from Waco   
490  album     Charley Crockett  country  Lil G.L. Presents: Jukebox Charley   
491  album  Muscadine Bloodline  country               Dispatch to 16th Ave.   
492  album      Little Big Town  country                             Mr. Sun   
493  album         Shelby Lynne  country                        Love, Shelby   

      año                      id  
0  

In [11]:
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(df_2018_2022_tracks)

        tipo                artista   genero  \
0    cancion   Florida Georgia Line  country   
1    cancion           Jordan Davis  country   
2    cancion           Jordan Davis  country   
3    cancion           Jordan Davis  country   
4    cancion         Tyler Childers  country   
..       ...                    ...      ...   
373  cancion       Charley Crockett  country   
374  cancion    Muscadine Bloodline  country   
375  cancion             Josh Meloy  country   
376  cancion             Josh Meloy  country   
377  cancion  Charles Wesley Godwin  country   

                                       nombre   año                      id  
0    Meant to Be (feat. Florida Georgia Line)  2018  4e4fqjx0Izh4svvTef1z7e  
1                 Slow Dance In A Parking Lot  2018  6MBUUSIWCzaXW4q58Ktrv9  
2                              Singles You Up  2018  4LiMQ6G9n84rDUBs9wtwz6  
3                             Take It From Me  2018  3dCZm5HrG0jIXAHfNkg2H0  
4                      Shake the 

In [ ]:
# Archivo para álbumes:

df_2018_2022_album.to_json(
    'country-extraccion-albumes.json',  # Nombre del archivo de salida
    orient='records',       # Formato legible (lista de registros)
    force_ascii=False,      # Permite caracteres especiales (ñ, á, é, etc.)
    indent=4                # Formato indentado para legibilidad
)

# Archivo para canciones:

df_2018_2022_tracks.to_json(
    'country-extraccion-tracks.json',  
    orient='records',      
    force_ascii=False,      
    indent=4  
)              